In [42]:
import pandas as pd

In [43]:
sales = pd.read_csv('all_flights_sales.csv', index_col=0)
products = pd.read_csv('../INPUT/one_ticket.csv')
schedule = pd.read_csv('../INPUT/data_fam_schedule.csv')

In [44]:
products = products[['Flight1', 'Flight2', 'Flight3', 'Sales']]
products[['Origin', 'Destination']] = None

In [45]:
sales.head(3)

,Flight_number,Leg,Origin,Destination,Depart_time,Arrive_time,Flight_time,B_class_sales,G_class_sales,H_class_sales,K_class_sales,L_class_sales,M_class_sales,Q_class_sales,S_class_sales,T_class_sales,V_class_sales,Y_class_sales,Total_sales
0,AA0001,AA0001EDBTFB,EDB,TFB,1155,1738,5.716667,8.03,4.43,8.96,10.46,70.50,11.51,6.54,3.37,12.67,32.38,66.45,235.30
1,AA0002,AA0002TFBEDB,TFB,EDB,2105,159,4.900000,13.37,3.80,24.52,13.21,83.25,7.83,11.98,1.21,0.95,52.75,27.92,240.79
2,AA0003,AA0003EDBTFB,EDB,TFB,2140,325,5.750000,4.33,5.63,17.01,13.79,15.31,8.24,28.76,0.70,2.97,21.98,27.33,146.05


In [46]:
products.head(3)

,Flight1,Flight2,Flight3,Sales,Origin,Destination
0,AA0001EDBTFB,.,.,206.45,None,None
1,AA0001EDBTFB,AA0059TFBKOV,.,4.55,None,None
2,AA0001EDBTFB,AA0067TFBKOV,.,0.75,None,None


In [47]:
# 获取起点终点
for i in range(len(products)):
    products.loc[i, 'Origin'] = products.loc[i, 'Flight1'][6:9]  # 起点
    if products.loc[i, 'Flight2'] == '.':  # 直飞航班终点
        products.loc[i, 'Destination'] = products.loc[i, 'Flight1'][9:]
        
    elif products.loc[i, 'Flight3'] == '.':  # 一次经停航班终点
        products.loc[i, 'Destination'] = products.loc[i, 'Flight2'][9:]
    
    else:  # 二次经停航班终点
        products.loc[i, 'Destination'] = products.loc[i, 'Flight3'][9:]

In [48]:
# 航班是否为经停航班，是1，否0
products['isStopover'] = None

for i in range(len(products)):
    if products.loc[i, 'Flight2'] != '.':
        products.loc[i, 'isStopover'] = 1
    else:
        products.loc[i, 'isStopover'] = 0

In [49]:
# 拼接起点终点
products['O_D'] = products['Origin'] + products['Destination']

In [50]:
products

,Flight1,Flight2,Flight3,Sales,Origin,Destination,isStopover,O_D
0,AA0001EDBTFB,.,.,206.45,EDB,TFB,0,EDBTFB
1,AA0001EDBTFB,AA0059TFBKOV,.,4.55,EDB,KOV,1,EDBKOV
2,AA0001EDBTFB,AA0067TFBKOV,.,0.75,EDB,KOV,1,EDBKOV
3,AA0001EDBTFB,AA0097TFBBOD,.,5.60,EDB,BOD,1,EDBBOD
4,AA0001EDBTFB,AA0097TFBBOD,AA0189BODGBJ,3.31,EDB,GBJ,1,EDBGBJ
...,...,...,...,...,...,...,...,...
4285,BA2644QEYFVH,.,.,22.36,QEY,FVH,0,QEYFVH
4286,BA2645QEYTNG,.,.,41.65,QEY,TNG,0,QEYTNG
4287,BA2645QEYTNG,AA0821TNGPHH,.,3.70,QEY,PHH,1,QEYPHH
4288,BA2670TFBZMX,.,.,68.36,TFB,ZMX,0,TFBZMX


In [51]:
stopover_sales, straight_sales = 0, 0

In [52]:
temp_df = products_temp[products_temp['O_D'] == 'EDBTFB']  # 取出某一起点终点组合航班
len(temp_df['isStopover'].drop_duplicates())

2

In [53]:
products_temp = products.copy()
checked = []
for i in range(len(products)):
    
    target = products.O_D[i]
    if target in checked:  # 不重复计算
        continue
        
    checked.append(target)
    temp_df = products_temp[products_temp['O_D'] == target]  # 取出某一起点终点组合航班
    temp_df = temp_df.reset_index()
    if len(temp_df) == 1:
        continue

    if len(temp_df['isStopover'].drop_duplicates()) != 2:
        continue
        
    for j in range(len(temp_df)):  # 分配销售量
        if temp_df.loc[j, 'isStopover'] == 1:
            stopover_sales += temp_df.Sales[j]
        else:
            straight_sales += temp_df.Sales[j]
        

In [54]:
f'stopover_sales:{stopover_sales}, straight_sales:{straight_sales}'

'stopover_sales:2534.8700000000003, straight_sales:46090.66999998199'